In [1]:
import random
import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

import clip
from PIL import Image
import torch.nn.functional as F

# from sklearn.metrics import classification_report
# from sklearn.preprocessing import MultiLabelBinarizer
# import torch.nn.functional as F


# from pathlib import Path
# from tqdm.notebook import tqdm
# from operator import itemgetter
# from sklearn.metrics import classification_report
# from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
k = 3
max_seq_length = 4096

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    #model_name="unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Data Preparation

In [3]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_pg_w_images.csv")

In [4]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,comics_title,comics_id,image_path
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...
...,...,...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
image_embed_model, preprocess = clip.load("ViT-B/32", device=device)

In [6]:
dataset_images = df.image_path.tolist()

In [7]:
image_encodings = {}

for image in tqdm(dataset_images):
    
    image_p = preprocess(Image.open(image)).unsqueeze(0).to(device) # type: ignore
    image_features = image_embed_model.encode_image(image_p).squeeze().cpu()
    image_encodings[image] = image_features.detach().numpy()

100%|██████████| 874/874 [04:34<00:00,  3.19it/s]


In [8]:
image_encodings['/Utilisateurs/umushtaq/emotion_analysis_comics/comics_dataset_images/001499/images/page00001.jpg']

array([-5.8887e-01,  6.4697e-01, -9.7961e-02,  9.7229e-02,  5.0879e-01,
       -1.8604e-01,  3.1641e-01, -2.4097e-01, -2.8549e-02, -2.8711e-01,
        2.0801e-01,  2.3041e-02,  3.8989e-01, -1.8417e-02, -2.8809e-01,
        2.1667e-01, -2.5952e-01, -1.4844e-01,  3.1891e-02, -3.3887e-01,
       -8.6572e-01, -2.9150e-01,  2.2144e-01,  7.7197e-01, -1.3562e-01,
        1.1218e-01,  8.1055e-01, -1.5076e-01, -9.9304e-02,  2.4704e-02,
       -2.3376e-01,  1.0687e-01, -7.6523e-03, -1.5759e-01,  5.0977e-01,
        6.7673e-03,  1.4380e-01,  5.2881e-01,  2.1545e-01, -3.3276e-01,
        2.8491e-01, -4.6411e-01,  8.2336e-02, -4.0234e-01,  3.2349e-01,
        1.3906e+00,  3.2495e-01, -2.1439e-02, -3.1641e-01, -9.2850e-03,
        1.3855e-01, -3.3521e-01,  1.9141e-01, -8.3801e-02,  1.6772e-01,
       -1.4819e-01,  2.4878e-01, -7.1472e-02,  1.8665e-01, -3.7750e-02,
        1.2183e-01,  7.3486e-02,  1.9495e-01,  2.1484e-01, -1.1475e-01,
        2.3645e-01,  2.0020e-01,  4.1675e-01,  6.9336e-02,  1.31

In [9]:
df['image_embedding'] = df.image_path.apply(lambda x: image_encodings[x])

In [10]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,comics_title,comics_id,image_path,image_embedding
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.589, 0.647, -0.09796, 0.0972, 0.509, -0.18..."
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.7397, 0.6367, -0.2167, 0.3684, 0.1387, -0...."
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.2688, 0.692, -0.2815, 0.002438, 0.3242, -0..."
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.4148, 0.504, 0.1909, 0.1763, 0.2874, 0.087..."
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",Jurassic League #4,1499,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.5005, 0.7563, -0.1572, 0.0667, 0.1208, 0.2..."
...,...,...,...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.3933, 0.706, -0.09546, 0.3477, -0.1542, -0..."
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.4753, 0.2815, -0.3374, 0.5576, 0.339, -0.2..."
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.5327, 0.2303, -0.2115, 0.709, 0.003029, -0..."
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",Stillwater #13,2200,/Utilisateurs/umushtaq/emotion_analysis_comics...,"[-0.6104, 0.5825, -0.0996, 0.522, 0.159, 0.101..."


### Get K neighbours

In [11]:
from operator import itemgetter

def get_k_neighbours(k, image, train_df, test_df):
    
    
    test_image_embedding = test_df[test_df.image_path == image]["image_embedding"].values[0]

    image_embed_d = {}
    
    for e in train_df.iterrows():
        
        embedding_key = tuple(e[1].image_embedding)
        if embedding_key not in image_embed_d:
            
            image_embed_d[e[1].image_path] = e[1].image_embedding

    train_image_paths = set(train_df.image_path)

    dist_l = []
    
    for p, v in image_embed_d.items():
        if p in train_image_paths:
            
            d = F.cosine_similarity(torch.tensor(test_image_embedding), torch.tensor(v), dim=0)
            dist_l.append((p, d.item()))

    sorted_dist_l = sorted(dist_l, key=itemgetter(1), reverse=True)
    
    return sorted_dist_l[0: k]

In [12]:
train_df = df[df.split == "TRAIN"].reset_index(drop=True)
test_df = df[df.split == "TEST"].reset_index(drop=True)

In [13]:
get_k_neighbours(3, test_df.iloc[3]['image_path'], train_df, test_df)

[('/Utilisateurs/umushtaq/emotion_analysis_comics/comics_dataset_images/001517/images/page00058.jpg',
  0.8876953125),
 ('/Utilisateurs/umushtaq/emotion_analysis_comics/comics_dataset_images/001517/images/page00056.jpg',
  0.88037109375),
 ('/Utilisateurs/umushtaq/emotion_analysis_comics/comics_dataset_images/001562/images/page00005.jpg',
  0.87158203125)]

In [14]:
import ast
import json

In [15]:
def prepare_similar_example_prompts(image, k, train_df, test_df, seed=33):
    """
    Create a part of the prompt made of k examples in the train set,
    whose topic is most similar to a given title.
    """

    random.seed(seed)
    
    neighbours_l = get_k_neighbours(k, image, train_df=train_df, test_df=test_df)
    #sampled_neighbours_l = random.sample(neighbours_l, 1)
    sampled_neighbours_l = neighbours_l

    prompt = ''
    cnt = 0
    
    for i, (image_path, dist) in enumerate(sampled_neighbours_l):
        
        example_df = train_df[train_df.image_path == image_path]
        sampled_rows = example_df.sample(1, random_state=42)

        
        
        
        for _, row in sampled_rows.iterrows():
            
            prompt += f'EXAMPLE {cnt + 1}\n\n'
            # Convert utterance string to a list
            utterance_list = json.loads(row.utterance) if row.utterance.strip().startswith('[') else ast.literal_eval(row.utterance)

            # Add numbered utterances to the prompt
            for idx, utt in enumerate(utterance_list, start=1):
                prompt += f'{idx}. {utt}\n'

            # Add emotion class for this row
            #prompt += f'\nEmotion class: {row.emotion_c}\n\n'
            prompt += f'\nOutput: {{"emotions": "{row.emotion_c}"}}\n\n'
            cnt += 1

        
        prompt += '\n'
        
    return prompt


In [16]:
print(prepare_similar_example_prompts(test_df.iloc[0]['image_path'], k, train_df, test_df, seed=33))

EXAMPLE 1

1. ROGER LEFT AFTER THAT.
2. OR RATHER, HIS PARENTS SENT HIM AWAY TO LIVE WITH SOME COUSIN OR OTHER.
3. I EXPECTED TO BE LET GO, BUT THE FAMILY WAS DESPERATE TO HIDE ANY HINT OF IMPROPRIETY AND TO SQUASH EVERY RUMOR.
4. THE DISMISSAL OF THE MAID AT THE SAME TIME AS ROGER'S DEPARTURE COULD BE THE CAUSE OF A GREAT DEAL OF TALK.
5. AND A GREAT DEAL OF TALK WAS ALWAYS TO BE AVOIDED.
6. SO I RESUMED MY CHORES, CLEANING HILL HOUSE FOR ALL THE GOOD MEN AND WOMEN WHO LIVED THERE.
7. ALL OF THOSE RICH, WELLBORN PEOPLE WHO EXPECTED A WELL-KEPT HOUSE.
8. FOR THEM, I SCRUBBED AND I WASHED AND I FOLDED AND I DUSTED AND I… AND I…
9. AND I HEARD ROGER'S WORDS, AGAIN AND AGAIN.
10. "DARLING,” HE SAID, "THE HUNT MUST END!"

Output: {"emotions": "[['anger', 'sadness'], ['sadness'], ['sadness'], ['sadness'], ['sadness'], ['fear', 'sadness'], ['fear', 'sadness'], ['fear', 'sadness'], ['fear', 'sadness'], ['sadness']]"}


EXAMPLE 2

1. AHA! LOOK THERE.
2. GREAT.
3. WHATTYA THINK, WE SHOULD INVES

In [17]:
### Prompt Preparatoin

In [18]:
# def build_sys_instruction():
    
#     emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Page-Level Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze all utterances on a comic book page.

# INPUT:
# - You will receive a list of utterances from a single page in a comic book
# - Each utterance may express one or multiple emotions
# - You will receive lists of utterances from {k} pages and their respective emotion classifications as example
# - Each example will contain multiple utterances and their respective emotion classifications
# - Given these examples, analyze the list of utterances following the same pattern

# TASK:
# - Carefully analyze the emotional context and tone of each utterance on the page
# - Identify applicable emotions from the following classes:
#    {formatted_classes}

# OUTPUT REQUIREMENTS:
# - STRICT JSON FORMAT IS MANDATORY
# - NO ADDITIONAL TEXT OR EXPLANATION IS ALLOWED
# - Example valid output: {{"emotions": [["anger"], ["fear", "sadness"], ["neutral"]]}}
# - Invalid formats will result in immediate rejection

# CRITICAL CONSTRAINTS:
# - Wrap EACH emotion in square brackets
# - Ensure valid JSON syntax at all times
# - Do NOT deviate from the specified JSON structure under any circumstances
# - Do NOT generate any additional text or explanation except the JSON object

# """
#     return instruction

In [19]:
# def build_sys_instruction():
#     emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Expert Comic Emotion Analyst Role

# You are a highly specialized emotion analysis expert with deep expertise in comic book dialogue interpretation and psychological assessment. Your task is to perform nuanced multi-label emotion classification for comic book dialogue.

# INPUT FORMAT:
# - You will receive sequential utterances from a single comic book page
# - Each utterance may express one or multiple emotions
# - You will be provided with {k} annotated pages as examples
# - Each example contains multiple utterances with their ground truth emotion labels
# - Follow these examples precisely for your analysis

# EMOTION CLASSIFICATION GUIDELINES:
# 1. Primary emotions ({formatted_classes}):
#    - Anger: hostility, frustration, rage, irritation
#    - Disgust: revulsion, contempt, disapproval
#    - Fear: anxiety, worry, terror, dread
#    - Sadness: grief, disappointment, melancholy
#    - Surprise: astonishment, shock, amazement
#    - Joy: happiness, excitement, pleasure, amusement
#    - Neutral: no clear emotional content

# 2. Analysis Principles:
#    - Consider both explicit emotional words and implicit context
#    - Account for punctuation and capitalization as emotional indicators
#    - Evaluate intensity markers (e.g., "!!!", CAPS, repeated letters)
#    - Factor in sequential context from surrounding utterances
#    - Default to "neutral" only when no clear emotion is present

# OUTPUT SPECIFICATIONS:
# - Strict JSON format: {{"emotions": [[emotion1, emotion2], [emotion3]]}}
# - The length of the value in JSON must EXACTLY match the number of utterances you have to classify
# - Do NOT generate any additional text or explanation except the JSON object
# - Arrays must be properly nested with correct brackets
# - NO ADDITIONAL TEXT OR EXPLANATION IS ALLOWED

# """
#     return instruction

In [20]:
def build_sys_instruction():
    emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Expert Comic Emotion Analyst Role

You are a highly specialized emotion analysis expert with deep expertise in comic book dialogue interpretation and psychological assessment. Your task is to perform nuanced multi-label emotion classification for comic book dialogue.

INPUT FORMAT:
- You will receive sequential utterances from a single comic book page
- Each utterance may express one or multiple emotions
- You will be provided with {k} annotated pages as examples
- Each example contains multiple utterances with their ground truth emotion labels
- Follow these examples precisely for your analysis

EMOTION CLASSIFICATION GUIDELINES:
1. Identify applicable emotions from the following classes: {formatted_classes}:
2. Analysis Principles:
   - Consider both explicit emotional words and implicit context
   - Account for punctuation and capitalization as emotional indicators
   - Evaluate intensity markers (e.g., "!!!", CAPS, repeated letters)
   - Factor in sequential context from surrounding utterances
   - Default to "neutral" only when no clear emotion is present

OUTPUT SPECIFICATIONS:
- Strict JSON format: {{"emotions": [[emotion1, emotion2], [emotion3]]}}
- The length of the value in JSON must EXACTLY match the number of utterances you have to classify
- Do NOT generate any additional text or explanation except the JSON object
- Arrays must be properly nested with correct brackets
- NO ADDITIONAL TEXT OR EXPLANATION IS ALLOWED

"""
    return instruction

In [21]:
# For LLaMA

#sys_msg_l = []
user_msg_l = []
task_msg_l = []

for _,row in tqdm(test_df.iterrows(), total=len(test_df)):
    
    utterance_list = json.loads(row.utterance) if row.utterance.strip().startswith('[') else ast.literal_eval(row.utterance)
    utterances = '\n'.join([f'{i+1}. {utt}' for i, utt in enumerate(utterance_list)])
    
    #sys_msg = {"role": "system", "content": build_sys_instruction()}
    user_msg = {"role":"user", "content": build_sys_instruction() + "\n\nEXAMPLES:\n\n" + prepare_similar_example_prompts(row.image_path, k, train_df=train_df, test_df=test_df) + f"INPUT:\n\nNow classify these utterances:\n\n{utterances}\n"}
    task_msg = {"role": "assistant", "content": f"\nOutput: "}
    
    #sys_msg_l.append(sys_msg)
    user_msg_l.append(user_msg)
    task_msg_l.append(task_msg)
    
    #break

100%|██████████| 156/156 [00:11<00:00, 13.34it/s]


In [22]:
#For LLaMA

prepared_sys_task_msg_l = []

for i in range(len(user_msg_l)):
    #prepared_sys_task_msg_l.append([sys_msg_l[i], user_msg_l[i], task_msg_l[i]])
    prepared_sys_task_msg_l.append([user_msg_l[i], task_msg_l[i]])

In [23]:
# # For Qwen

# prepared_sys_task_msg_l = []

# for i in range(len(user_msg_l)):
#     prepared_sys_task_msg_l.append([user_msg_l[i], task_msg_l[i]])

In [24]:
len(prepared_sys_task_msg_l)

156

In [25]:
prepared_sys_task_msg_l[0]

[{'role': 'user',
  'content': '### Expert Comic Emotion Analyst Role\n\nYou are a highly specialized emotion analysis expert with deep expertise in comic book dialogue interpretation and psychological assessment. Your task is to perform nuanced multi-label emotion classification for comic book dialogue.\n\nINPUT FORMAT:\n- You will receive sequential utterances from a single comic book page\n- Each utterance may express one or multiple emotions\n- You will be provided with 3 annotated pages as examples\n- Each example contains multiple utterances with their ground truth emotion labels\n- Follow these examples precisely for your analysis\n\nEMOTION CLASSIFICATION GUIDELINES:\n1. Identify applicable emotions from the following classes: "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral":\n2. Analysis Principles:\n   - Consider both explicit emotional words and implicit context\n   - Account for punctuation and capitalization as emotional indicators\n   - Evaluate intensi

In [26]:
### Inference

In [27]:
def batch_tensor(tensor, batch_size):
    return [tensor[i:i+batch_size] for i in range(0, tensor.size(0), batch_size)]

In [88]:
inputs = tokenizer.apply_chat_template(
            prepared_sys_task_msg_l,
            #pad_token = inference_tokenizer.bos_token,
            padding=True,
            truncation=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
)

In [89]:
inputs.input_ids[0]

tensor([128004, 128004, 128004,  ...,  78191, 128007,    271])

In [90]:
print(tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True))

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

user

### Expert Comic Emotion Analyst Role

You are a highly specialized emotion analysis expert with deep expertise in comic book dialogue interpretation and psychological assessment. Your task is to perform nuanced multi-label emotion classification for comic book dialogue.

INPUT FORMAT:
- You will receive sequential utterances from a single comic book page
- Each utterance may express one or multiple emotions
- You will be provided with 3 annotated pages as examples
- Each example contains multiple utterances with their ground truth emotion labels
- Follow these examples precisely for your analysis

EMOTION CLASSIFICATION GUIDELINES:
1. Identify applicable emotions from the following classes: "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral":
2. Analysis Principles:
   - Consider both explicit emotional words and implicit context
   - Account for punctuation and capitalization as emotional ind

In [91]:
BATCH_SIZE = 8

input_ids_batches = batch_tensor(inputs['input_ids'], BATCH_SIZE) # type: ignore
attention_mask_batches = batch_tensor(inputs['attention_mask'], BATCH_SIZE) # type: ignore

In [92]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

In [106]:
generated_outputs = []

for i, (input_ids_batch, attention_mask_batch) in tqdm(enumerate(zip(input_ids_batches, attention_mask_batches)), total=len(input_ids_batches)):
    
    #print(f"\n ***** Processing batch {i + 1} *****\n")
    
    if torch.any(torch.isnan(input_ids_batch)) or torch.any(torch.isinf(input_ids_batch)): # type: ignore
        print("Invalid input_ids detected")

    if torch.any(torch.isnan(attention_mask_batch)) or torch.any(torch.isinf(attention_mask_batch)): # type: ignore
        print("Invalid attention_mask detected")

    
    inputs = {
        'input_ids': input_ids_batch.to(model.device), # type: ignore
        'attention_mask': attention_mask_batch.to(model.device) # type: ignore
    }

    outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    #eos_token_id=terminators,
    do_sample=True,
    num_beams=1,
    temperature=0.7,
    top_p=1.0,
    )
    
    for output in outputs:
        generated_outputs.append(tokenizer.decode(output[inputs['input_ids'].shape[1]:], skip_special_tokens=True))
    
    
    #generated_outputs.append(outputs)
    
    #generated_outputs.append(inference_tokenizer.decode(outputs[inputs['input_ids'].shape[1]:]))
    #generated_outputs.append(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))
    #
    # generated_outputs.extend(tokenizer.batch_decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))
    #break

100%|██████████| 20/20 [02:17<00:00,  6.88s/it]


In [206]:
len(generated_outputs)

156

In [207]:
generated_outputs

['{"emotions": "[[\'joy\'], [\'neutral\'], [\'joy\'], [\'joy\'], [\'neutral\'], [\'joy\'], [\'fear\'], [\'neutral\'], [\'joy\'], [\'sadness\', \'joy\']]"',
 '{"emotions": "[[\'joy\',\'surprise\'], [\'sadness\', \'anger\', \'fear\'], [\'anger\', \'disgust\'], [\'sadness\', \'anger\', \'disgust\'], [\'joy\',\'surprise\'], [\'anger\', \'fear\'], [\'fear\',\'sadness\'], [\'sadness\', \'anger\']]"',
 '{"emotions": "[[\'anger\', \'fear\'], [\'fear\', \'disgust\'], [\'anger\'], [\'fear\',\'surprise\'], [\'anger\',\'sadness\'], [\'anger\', \'fear\'], [\'fear\', \'disgust\'], [\'sadness\', \'joy\'], [\'fear\', \'disgust\'], [\'surprise\'], [\'anger\', \'fear\'], [\'anger\'], [\'fear\',\'surprise\'], [\'fear\', \'joy\'], [\'neutral\']]"}\n\nor',
 '{"emotions": "[[\'anger\', \'fear\', \'disgust\'], [\'anger\'], [\'anger\'], [\'neutral\'], [\'sadness\', \'anger\'], [\'anger\', \'fear\'], [\'anger\'], [\'anger\',\'sadness\'], [\'anger\'], [\'anger\'], [\'anger\', \'disgust\', \'fear\'], [\'anger\',

In [208]:
#generated_outputs = [x.split("\n\n")[0] for x in generated_outputs]

In [209]:
len(generated_outputs)

156

In [210]:
generated_outputs

['{"emotions": "[[\'joy\'], [\'neutral\'], [\'joy\'], [\'joy\'], [\'neutral\'], [\'joy\'], [\'fear\'], [\'neutral\'], [\'joy\'], [\'sadness\', \'joy\']]"',
 '{"emotions": "[[\'joy\',\'surprise\'], [\'sadness\', \'anger\', \'fear\'], [\'anger\', \'disgust\'], [\'sadness\', \'anger\', \'disgust\'], [\'joy\',\'surprise\'], [\'anger\', \'fear\'], [\'fear\',\'sadness\'], [\'sadness\', \'anger\']]"',
 '{"emotions": "[[\'anger\', \'fear\'], [\'fear\', \'disgust\'], [\'anger\'], [\'fear\',\'surprise\'], [\'anger\',\'sadness\'], [\'anger\', \'fear\'], [\'fear\', \'disgust\'], [\'sadness\', \'joy\'], [\'fear\', \'disgust\'], [\'surprise\'], [\'anger\', \'fear\'], [\'anger\'], [\'fear\',\'surprise\'], [\'fear\', \'joy\'], [\'neutral\']]"}\n\nor',
 '{"emotions": "[[\'anger\', \'fear\', \'disgust\'], [\'anger\'], [\'anger\'], [\'neutral\'], [\'sadness\', \'anger\'], [\'anger\', \'fear\'], [\'anger\'], [\'anger\',\'sadness\'], [\'anger\'], [\'anger\'], [\'anger\', \'disgust\', \'fear\'], [\'anger\',

In [211]:
import json_repair


In [212]:
outputs_processed = [json_repair.loads(output) for output in generated_outputs]

In [213]:
len(outputs_processed)

156

In [214]:
outputs_processed

[{'emotions': "[['joy'], ['neutral'], ['joy'], ['joy'], ['neutral'], ['joy'], ['fear'], ['neutral'], ['joy'], ['sadness', 'joy']]"},
 {'emotions': "[['joy','surprise'], ['sadness', 'anger', 'fear'], ['anger', 'disgust'], ['sadness', 'anger', 'disgust'], ['joy','surprise'], ['anger', 'fear'], ['fear','sadness'], ['sadness', 'anger']]"},
 {'emotions': "[['anger', 'fear'], ['fear', 'disgust'], ['anger'], ['fear','surprise'], ['anger','sadness'], ['anger', 'fear'], ['fear', 'disgust'], ['sadness', 'joy'], ['fear', 'disgust'], ['surprise'], ['anger', 'fear'], ['anger'], ['fear','surprise'], ['fear', 'joy'], ['neutral']]"},
 {'emotions': "[['anger', 'fear', 'disgust'], ['anger'], ['anger'], ['neutral'], ['sadness', 'anger'], ['anger', 'fear'], ['anger'], ['anger','sadness'], ['anger'], ['anger'], ['anger', 'disgust', 'fear'], ['anger', 'disgust'], ['joy', 'anger'], ['anger', 'disgust'], ['anger', 'fear'], ['anger', 'disgust'], ['neutral'], ['joy'], ['anger', 'fear'], ['anger']]"},
 {'emotion

In [215]:
grounds = test_df.emotion_c.tolist()

In [216]:
grounds

["[['surprise', 'joy'], ['joy'], ['surprise', 'joy'], ['joy'], ['joy'], ['joy'], ['surprise'], ['joy'], ['joy'], ['neutral'], ['neutral']]",
 "[['neutral'], ['neutral'], ['anger', 'disgust'], ['anger', 'disgust'], ['neutral'], ['sadness'], ['sadness']]",
 "[['anger', 'sadness'], ['anger', 'sadness'], ['anger', 'sadness'], ['fear', 'surprise'], ['surprise'], ['joy'], ['anger', 'surprise'], ['joy'], ['joy'], ['joy'], ['anger'], ['anger'], ['surprise', 'joy'], ['fear', 'sadness'], ['fear', 'sadness'], ['fear', 'surprise']]",
 "[['anger', 'disgust'], ['anger', 'disgust'], ['anger', 'disgust'], ['fear', 'sadness'], ['fear', 'sadness', 'surprise'], ['sadness'], ['sadness'], ['fear', 'sadness'], ['sadness', 'surprise'], ['sadness', 'surprise'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger', 'disgust'], ['joy'], ['joy'], ['surprise', 'joy'], ['surprise', 'joy'], ['anger', 'surprise'], ['anger', 'surprise']]",
 "[['neutral'], ['joy'], ['joy']]",
 "[['neutral'], ['fear'], ['neutral'], ['ang

In [217]:
predictions = []
bad_idx = []

for i, obj in enumerate(outputs_processed):
    try:
        predictions.append(obj['emotions'])
    except:
        print(i)
        bad_idx.append(i)

48
123


In [218]:
grounds = [label for i, label in enumerate(grounds) if i not in bad_idx]

In [219]:
len(grounds), len(predictions)

(154, 154)

In [220]:
grounds_l = [ast.literal_eval(item) for item in grounds]

In [221]:
predictions

["[['joy'], ['neutral'], ['joy'], ['joy'], ['neutral'], ['joy'], ['fear'], ['neutral'], ['joy'], ['sadness', 'joy']]",
 "[['joy','surprise'], ['sadness', 'anger', 'fear'], ['anger', 'disgust'], ['sadness', 'anger', 'disgust'], ['joy','surprise'], ['anger', 'fear'], ['fear','sadness'], ['sadness', 'anger']]",
 "[['anger', 'fear'], ['fear', 'disgust'], ['anger'], ['fear','surprise'], ['anger','sadness'], ['anger', 'fear'], ['fear', 'disgust'], ['sadness', 'joy'], ['fear', 'disgust'], ['surprise'], ['anger', 'fear'], ['anger'], ['fear','surprise'], ['fear', 'joy'], ['neutral']]",
 "[['anger', 'fear', 'disgust'], ['anger'], ['anger'], ['neutral'], ['sadness', 'anger'], ['anger', 'fear'], ['anger'], ['anger','sadness'], ['anger'], ['anger'], ['anger', 'disgust', 'fear'], ['anger', 'disgust'], ['joy', 'anger'], ['anger', 'disgust'], ['anger', 'fear'], ['anger', 'disgust'], ['neutral'], ['joy'], ['anger', 'fear'], ['anger']]",
 "[['fear', 'anger'], ['joy'], ['joy']]",
 "[['joy', 'neutral'], [

In [222]:
predictions[17]

"[['joy'], ['joy'], ['anger', 'joy'], ['anger'], ['joy', 'anger'], ['anger','sadness'], ['joy','sadness'], ['joy', 'anger'], ['anger','surprise'], ['anger'], ['anger','sadness'], ['anger', 'joy'], ['joy']]"

In [223]:
predictions_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:
        predictions_l.append(ast.literal_eval(pred))
    except:
        print(i)
        bad_idx.append(i)

#predictions_l = [ast.literal_eval(item) for item in predictions]

149


In [224]:
len(grounds)

154

In [225]:
bad_idx

[149]

In [228]:
grounds_l = [label for i, label in enumerate(grounds_l) if i not in bad_idx]
#predictions_l = [label for i, label in enumerate(predictions_l) if i not in bad_idx]

In [229]:
len(grounds_l)

153

In [230]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds_l, predictions_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

0 11 10
1 7 8
2 16 15
3 21 20
5 20 17
6 15 17
9 11 13
14 23 22
15 23 21
17 12 13
19 14 13
24 13 12
25 13 12
30 6 5
36 17 16
43 8 10
44 7 8
45 13 12
46 11 13
54 8 9
55 7 9
57 6 7
61 7 8
63 7 8
64 8 10
68 6 5
75 7 8
76 6 7
78 8 9
79 8 9
81 10 11
84 13 12
85 10 9
86 15 16
87 10 9
90 8 9
93 6 7
97 8 10
98 13 19
102 13 10
105 5 6
108 4 5
109 6 7
111 6 7
116 9 8
117 12 11
118 13 12
119 17 18
120 19 17
121 13 12
127 12 11
129 8 9
130 11 12
137 6 7
142 4 3
143 12 13
146 16 18
147 11 10
148 11 10
149 7 10
150 11 12


In [231]:
for idx, (i,j) in enumerate(zip(grounds_l, predictions_l)):
    if len(i) > len(j):
        grounds_l[idx] = grounds_l[idx][:len(j)]
    elif len(j) > len(i):
        predictions_l[idx] = predictions_l[idx][:len(i)]
    

In [232]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds_l, predictions_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

In [233]:
# grounds = [label for i, label in enumerate(grounds_l) if i not in bad_idx]
# predictions = [label for i, label in enumerate(predictions_l) if i not in bad_idx]

In [234]:
len(grounds_l), len(predictions_l)

(153, 153)

In [235]:
grounds = [item for sublist in grounds_l for item in sublist]
predictions = [item for sublist in predictions_l for item in sublist]

In [236]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [237]:
mlb = MultiLabelBinarizer()

In [238]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['angry', 'appreciation', 'concern', 'confidence', 'confusion', 'curiosity', 'gratitude', 'hope', 'neutrality', 'relief', 'shame', 'urgency'] will be ignored
  warnings.warn(


In [239]:
y_true_mhot.shape, y_pred_mhot.shape

((1267, 7), (1267, 7))

In [240]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.492     0.649     0.560       427
     disgust      0.100     0.200     0.133        50
        fear      0.289     0.540     0.377       285
         joy      0.467     0.390     0.425       290
     neutral      0.167     0.170     0.168       106
     sadness      0.440     0.460     0.450       326
    surprise      0.371     0.374     0.373       342

   micro avg      0.381     0.465     0.419      1826
   macro avg      0.332     0.398     0.355      1826
weighted avg      0.395     0.465     0.421      1826
 samples avg      0.383     0.466     0.398      1826



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
